In [7]:
# Problem Set 4B
# Name: Alon Parag
# Collaborators:
# Work started at 24.12.2020 14:29
# Work finished at 27.12.2020 at 12:00
# Use pytest from ps4b folder to run unit tests
# view dubgging processin debugging.ipynb
import string

### HELPER CODE ###
def load_words(file_name):
    '''
    file_name (string): the name of the file containing 
    the list of words to load    
    
    Returns: a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    '''
    print("Loading word list from file...")
    # inFile: file
    inFile = open(file_name, 'r')
    # wordlist: list of strings
    wordlist = []
    for line in inFile:
        wordlist.extend([word.lower() for word in line.split(' ')])
    print("  ", len(wordlist), "words loaded.")
    return wordlist

def is_word(word_list, word):
    '''
    Determines if word is a valid word, ignoring
    capitalization and punctuation

    word_list (list): list of words in the dictionary.
    word (string): a possible word.
    
    Returns: True if word is in word_list, False otherwise

    Example:
    >>> is_word(word_list, 'bat') returns
    True
    >>> is_word(word_list, 'asdf') returns
    False
    '''
    word = word.lower()
    word = word.strip(" !@#$%^&*()-_+={}[]|\\:;'<>?,./\"")
    return word in word_list

def get_story_string():
    """
    Returns: a story in encrypted text.
    """
    f = open("story.txt", "r")
    story = str(f.read())
    f.close()
    return story

### END HELPER CODE ###

WORDLIST_FILENAME = 'words.txt'

class Message(object):
    def __init__(self, text):
        '''
        Initializes a Message object
                
        text (string): the message's text

        a Message object has two attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
        '''
        self.__message_text = text
        self.__valid_words = []
        for word in text.split():
            if is_word(word_list, word): self.__valid_words.append(word)

    def get_message_text(self):
        '''
        Used to safely access self.message_text outside of the class
        
        Returns: self.message_text
        '''
        return self.__message_text

    def get_valid_words(self):
        '''
        Used to safely access a copy of self.valid_words outside of the class.
        This helps you avoid accidentally mutating class attributes.
        
        Returns: a COPY of self.valid_words
        '''
        return self.__valid_words[:]

    def build_shift_dict(self, shift):
        '''
        Creates a dictionary that can be used to apply a cipher to a letter.
        The dictionary maps every uppercase and lowercase letter to a
        character shifted down the alphabet by the input shift. The dictionary
        should have 52 keys of all the uppercase letters and all the lowercase
        letters only.        
        
        shift (integer): the amount by which to shift every letter of the 
        alphabet. 0 <= shift < 26

        Returns: a dictionary mapping a letter (string) to 
                 another letter (string). 
        '''
        try:
            assert shift>= 0 and shift < 26
            assert type(shift) == int
        except AssertionError:
            print('Error, Shift key must be an integer from 0 to 26 excluding 26')
        except:
            print('An error has occured with the shift type')
        else:
            base_dict = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y',25:'Z',26:'a',27:'b',28:'c',29:'d',30:'e',31:'f',32:'g',33:'h',34:'i',35:'j',36:'k',37:'l',38:'m',39:'n',40:'o',41:'p',42:'q',43:'r',44:'s',45:'t',46:'u',47:'v',48:'w',49:'x',50:'y',51:'z', }
            shifted_dict = {}
            for key in base_dict.keys():
                if key < 26:
                    if key - shift < 0:
                        shifted_dict[base_dict[key]] = base_dict[26 - (abs(key - shift))]
                    else:
                        shifted_dict[base_dict[key]] = base_dict[key-shift]
                else:
                    if key - shift < 26:
                        shifted_dict[base_dict[key]] = base_dict[52-abs(key - shift-26)]
                    else:
                        shifted_dict[base_dict[key]] = base_dict[key - shift]
            return shifted_dict.copy()

    def apply_shift(self, shift):
        '''
        Applies the Caesar Cipher to self.message_text with the input shift.
        Creates a new string that is self.message_text shifted down the
        alphabet by some number of characters determined by the input shift        
        
        shift (integer): the shift with which to encrypt the message.
        0 <= shift < 26

        Returns: the message text (string) in which every character is shifted
             down the alphabet by the input shift
        '''
        shift_dict = self.build_shift_dict(shift)
        assert type(shift_dict) == dict
        encrypted_str = ''
        for key in self.get_message_text():
            if key in shift_dict.keys():
                encrypted_str+=shift_dict[key]
            elif not key in string.ascii_letters:
                encrypted_str+= key
        
        return encrypted_str

class PlaintextMessage(Message):
    def __init__(self, text, shift):
        '''
        Initializes a PlaintextMessage object        
        
        text (string): the message's text
        shift (integer): the shift associated with this message

        A PlaintextMessage object inherits from Message and has five attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
            self.shift (integer, determined by input shift)
            self.encryption_dict (dictionary, built using shift)
            self.message_text_encrypted (string, created using shift)

        '''
        Message.__init__(self, text)
        self.__shift = shift
        self.__encryption_dict = self.build_shift_dict(shift)
        self.__message_text_encrypted = self.apply_shift(shift)

    def get_shift(self):
        '''
        Used to safely access self.shift outside of the class
        
        Returns: self.shift
        '''
        return self.__shift

    def get_encryption_dict(self):
        '''
        Used to safely access a copy self.encryption_dict outside of the class
        
        Returns: a COPY of self.encryption_dict
        '''
        return self.__encryption_dict

    def get_message_text_encrypted(self):
        '''
        Used to safely access self.message_text_encrypted outside of the class
        
        Returns: self.message_text_encrypted
        '''
        return self.__message_text_encrypted

    def change_shift(self, shift):
        '''
        Changes self.shift of the PlaintextMessage and updates other 
        attributes determined by shift.        
        
        shift (integer): the new shift that should be associated with this message.
        0 <= shift < 26

        Returns: nothing
        '''
        self.__shift = shift
        self.__encryption_dict = self.build_shift_dict(shift)
        self.__message_text_encrypted = self.apply_shift(shift)
        assert self.get_shift() == shift
        assert self.get_message_text_encrypted() == self.apply_shift(shift)
        assert self.get_encryption_dict() == self.build_shift_dict(shift)


class CiphertextMessage(Message):
    def __init__(self, text):
        '''
        Initializes a CiphertextMessage object
                
        text (string): the message's text

        a CiphertextMessage object has two attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
        '''
        Message.__init__(self, text)

    def decrypt_message(self):
        '''
        Decrypt self.message_text by trying every possible shift value
        and find the "best" one. We will define "best" as the shift that
        creates the maximum number of real words when we use apply_shift(shift)
        on the message text. If s is the original shift value used to encrypt
        the message, then we would expect 26 - s to be the best shift value 
        for decrypting it.

        Note: if multiple shifts are equally good such that they all create 
        the maximum number of valid words, you may choose any of those shifts 
        (and their corresponding decrypted messages) to return

        Returns: a tuple of the best shift value used to decrypt the message
        and the decrypted message text using that shift value
        '''
        # initialize a list of possible decryptions as list_decryptions
        list_decryptions = []
        # loop through all 26 decryption shifts
        for i in range(26):
        #     store shift value in shift_value
            shift_value = 25 if i == 0 else 26-i
        # Create a PlaintextMessage with current shift
            currentMessage = Message(self.apply_shift(shift_value))
        #     store decrypted message in decrypted_message
            decrypted_message = currentMessage.get_message_text()
        #     store real words in words_count
            words_counter = len(currentMessage.get_valid_words())
        #     append a tuple of (words_count,(shift_value, decrypted_message)) in list_decryptions
            list_decryptions.append((words_counter, (i, decrypted_message)))
        # sort list_decryptions
        list_decryptions.sort(reverse = True)
        # print(list_decryptions)
        return list_decryptions[0][1]

word_list =load_words('./words.txt')
if __name__ == '__main__':
    word_list =load_words('./words.txt')
#    #Example test case (PlaintextMessage)
#    plaintext = PlaintextMessage('hello', 2)
#    print('Expected Output: jgnnq')
#    print('Actual Output:', plaintext.get_message_text_encrypted())
#
#    #Example test case (CiphertextMessage)
#    ciphertext = CiphertextMessage('jgnnq')
#    print('Expected Output:', (24, 'hello'))
#    print('Actual Output:', ciphertext.decrypt_message())

    #TODO: WRITE YOUR TEST CASES HERE

    #TODO: best shift value and unencrypted story 
    
    pass #delete this line and replace with your code here

test_1 = CiphertextMessage('jgnnq')
test_2 = CiphertextMessage('V xpxphwzm rdoc hzydxdivg kmjkzmodzn')
test_3 = CiphertextMessage('Kzgml, kzgml, dwl al sdd gml')
print(test_1.decrypt_message())
print(test_2.decrypt_message())
print(test_3.decrypt_message())



Loading word list from file...
   55901 words loaded.
Loading word list from file...
   55901 words loaded.
(24, 'hello')
(5, 'A cucumber with medicinal properties')
(8, 'Shout, shout, let it all out')


In [1]:
# Problem Set 4B
# Name: Alon Parag
# Collaborators:
#Work started at 24.12.2020 14:29
# Time Spent: x:xx

import string

### HELPER CODE ###
def load_words(file_name):
    '''
    file_name (string): the name of the file containing 
    the list of words to load    
    
    Returns: a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    '''
    print("Loading word list from file...")
    # inFile: file
    inFile = open(file_name, 'r')
    # wordlist: list of strings
    wordlist = []
    for line in inFile:
        wordlist.extend([word.lower() for word in line.split(' ')])
    print("  ", len(wordlist), "words loaded.")
    return wordlist

def is_word(word_list, word):
    '''
    Determines if word is a valid word, ignoring
    capitalization and punctuation

    word_list (list): list of words in the dictionary.
    word (string): a possible word.
    
    Returns: True if word is in word_list, False otherwise

    Example:
    >>> is_word(word_list, 'bat') returns
    True
    >>> is_word(word_list, 'asdf') returns
    False
    '''
    word = word.lower()
    word = word.strip(" !@#$%^&*()-_+={}[]|\\:;'<>?,./\"")
    return word in word_list

def get_story_string():
    """
    Returns: a story in encrypted text.
    """
    f = open("story.txt", "r")
    story = str(f.read())
    f.close()
    return story

### END HELPER CODE ###

WORDLIST_FILENAME = 'words.txt'

class Message(object):
    def __init__(self, text):
        '''
        Initializes a Message object
                
        text (string): the message's text

        a Message object has two attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
        '''
        self.__message_text = text
        self.__valid_words = []
        for word in text.split():
            if is_word(word_list, word): self.__valid_words.append(word)

    def get_message_text(self):
        '''
        Used to safely access self.message_text outside of the class
        
        Returns: self.message_text
        '''
        return self.__message_text

    def get_valid_words(self):
        '''
        Used to safely access a copy of self.valid_words outside of the class.
        This helps you avoid accidentally mutating class attributes.
        
        Returns: a COPY of self.valid_words
        '''
        return self.__valid_words[:]

    def build_shift_dict(self, shift):
        '''
        Creates a dictionary that can be used to apply a cipher to a letter.
        The dictionary maps every uppercase and lowercase letter to a
        character shifted down the alphabet by the input shift. The dictionary
        should have 52 keys of all the uppercase letters and all the lowercase
        letters only.        
        
        shift (integer): the amount by which to shift every letter of the 
        alphabet. 0 <= shift < 26

        Returns: a dictionary mapping a letter (string) to 
                 another letter (string). 
        '''
        try:
            assert shift>= 0 and shift < 26
            assert type(shift) == int
        except AssertionError:
            print('Error, Shift key must be an integer from 0 to 26 excluding 26')
        except:
            print('An error has occured with the shift type')
        else:
            base_dict = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y',25:'Z',26:'a',27:'b',28:'c',29:'d',30:'e',31:'f',32:'g',33:'h',34:'i',35:'j',36:'k',37:'l',38:'m',39:'n',40:'o',41:'p',42:'q',43:'r',44:'s',45:'t',46:'u',47:'v',48:'w',49:'x',50:'y',51:'z', }
            shifted_dict = {}
            for key in base_dict.keys():
                if key < 26:
                    if key - shift < 0:
                        shifted_dict[base_dict[key]] = base_dict[26 - (abs(key - shift))]
                    else:
                        shifted_dict[base_dict[key]] = base_dict[key-shift]
                else:
                    if key - shift < 26:
                        shifted_dict[base_dict[key]] = base_dict[52-abs(key - shift-26)]
                    else:
                        shifted_dict[base_dict[key]] = base_dict[key - shift]
            return shifted_dict.copy()

    def apply_shift(self, shift):
        '''
        Applies the Caesar Cipher to self.message_text with the input shift.
        Creates a new string that is self.message_text shifted down the
        alphabet by some number of characters determined by the input shift        
        
        shift (integer): the shift with which to encrypt the message.
        0 <= shift < 26

        Returns: the message text (string) in which every character is shifted
             down the alphabet by the input shift
        '''
        shift_dict = self.build_shift_dict(shift)
        assert type(shift_dict) == dict
        encrypted_str = ''
        for key in self.get_message_text():
            if key in shift_dict.keys():
                encrypted_str+=shift_dict[key]
            elif not key in string.ascii_letters:
                encrypted_str+= key
        
        return encrypted_str

class PlaintextMessage(Message):
    def __init__(self, text, shift):
        '''
        Initializes a PlaintextMessage object        
        
        text (string): the message's text
        shift (integer): the shift associated with this message

        A PlaintextMessage object inherits from Message and has five attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
            self.shift (integer, determined by input shift)
            self.encryption_dict (dictionary, built using shift)
            self.message_text_encrypted (string, created using shift)

        '''
        Message.__init__(self, text)
        self.__shift = shift
        self.__encryption_dict = self.build_shift_dict(shift)
        self.__message_text_encrypted = self.apply_shift(shift)

    def get_shift(self):
        '''
        Used to safely access self.shift outside of the class
        
        Returns: self.shift
        '''
        return self.__shift

    def get_encryption_dict(self):
        '''
        Used to safely access a copy self.encryption_dict outside of the class
        
        Returns: a COPY of self.encryption_dict
        '''
        return self.__encryption_dict

    def get_message_text_encrypted(self):
        '''
        Used to safely access self.message_text_encrypted outside of the class
        
        Returns: self.message_text_encrypted
        '''
        return self.__message_text_encrypted

    def change_shift(self, shift):
        '''
        Changes self.shift of the PlaintextMessage and updates other 
        attributes determined by shift.        
        
        shift (integer): the new shift that should be associated with this message.
        0 <= shift < 26

        Returns: nothing
        '''
        self.__shift = shift
        self.__encryption_dict = self.build_shift_dict(shift)
        self.__message_text_encrypted = self.apply_shift(shift)
        assert self.get_shift() == shift
        assert self.get_message_text_encrypted() == self.apply_shift(shift)
        assert self.get_encryption_dict() == self.build_shift_dict(shift)


class CiphertextMessage(Message):
    def __init__(self, text):
        '''
        Initializes a CiphertextMessage object
                
        text (string): the message's text

        a CiphertextMessage object has two attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
        '''
        Message.__init__(self, text)

    def decrypt_message(self):
        '''
        Decrypt self.message_text by trying every possible shift value
        and find the "best" one. We will define "best" as the shift that
        creates the maximum number of real words when we use apply_shift(shift)
        on the message text. If s is the original shift value used to encrypt
        the message, then we would expect 26 - s to be the best shift value 
        for decrypting it.

        Note: if multiple shifts are equally good such that they all create 
        the maximum number of valid words, you may choose any of those shifts 
        (and their corresponding decrypted messages) to return

        Returns: a tuple of the best shift value used to decrypt the message
        and the decrypted message text using that shift value
        '''
        # initialize a list of possible decryptions as list_decryptions
        list_decryptions = []
        # loop through all 26 decryption shifts
        for i in range(26):
        #     store shift value in shift_value
            shift_value = 25 if i == 0 else 26-i
        # Create a PlaintextMessage with current shift
            currentMessage = Message(self.apply_shift(shift_value))
        #     store decrypted message in decrypted_message
            decrypted_message = currentMessage.get_message_text()
        #     store real words in words_count
            words_counter = len(currentMessage.get_valid_words())
        #     append a tuple of (words_count,(shift_value, decrypted_message)) in list_decryptions
            list_decryptions.append((words_counter, (i, decrypted_message)))
        # sort list_decryptions
        list_decryptions.sort()
        print(list_decryptions)
        # print(list_decryptions[0][1])
        return list_decryptions[0][1]

word_list =load_words('./words.txt')
if __name__ == '__main__':
    word_list =load_words('./words.txt')
#    #Example test case (PlaintextMessage)
#    plaintext = PlaintextMessage('hello', 2)
#    print('Expected Output: jgnnq')
#    print('Actual Output:', plaintext.get_message_text_encrypted())
#
#    #Example test case (CiphertextMessage)
#    ciphertext = CiphertextMessage('jgnnq')
#    print('Expected Output:', (24, 'hello'))
#    print('Actual Output:', ciphertext.decrypt_message())

    #TODO: WRITE YOUR TEST CASES HERE

    #TODO: best shift value and unencrypted story 
    
    pass #delete this line and replace with your code here

test_1 = Message('Hello dfghj')
# test_1.get_valid_words()

Loading word list from file...
   55901 words loaded.
Loading word list from file...
   55901 words loaded.


['Hello']